# Prática de Aprendizado Supervisionado

**Importando bibliotecas e funções**

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.io.arff import loadarff
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import time
import warnings

warnings.filterwarnings("ignore")

**Lendo o arquivo**

In [4]:
def leitura(dataset, nome):

    # Carrega o .arff
    raw_data = loadarff('../Prática 1/Extraidos/%s/%s.arff' % (dataset, nome))
    # Transforma o .arff em um Pandas Dataframe
    return pd.DataFrame(raw_data[0])
    # Imprime o Dataframe com suas colunas

**Separando em Conjunto de Treino e Teste**

In [6]:
# Com o iloc voce retira as linhas e colunas que quiser do Dataframe, no caso aqui sem as classes
def treinoTeste(df):

    X = df.iloc[:, 0:-1].values

    # Aqui salvamos apenas as classes agora
    y = df['class']
    # Substituimos os valores binários por inteiro
    bow = []
    int_value = 0
    y_aux = []
    for i in y:
      if i in bow:
        y_aux.append(int_value)
      else:
        bow.append(i)
        int_value += 1
        y_aux.append(int_value)
    # Novo y
    y = y_aux
    
    return train_test_split(X,y,test_size=0.2,random_state=327)

**Padronizando os dados com Técnicas de Normalização**

In [7]:
def normalizar(X_train, X_test, selectedNormalization):

    if selectedNormalization == 0:
        return X_train, X_test
    if selectedNormalization == 1:
        scaler = preprocessing.MinMaxScaler()
    if selectedNormalization == 2:
        scaler = preprocessing.StandardScaler()
    if selectedNormalization == 3:
        scaler = preprocessing.MaxAbsScaler()
    if selectedNormalization == 4:
        scaler = preprocessing.RobustScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test

**Treinando os Classificadores**

In [8]:
def treinarClassificadores(classificador, X_train, X_test, y_train, y_test):
        
    cls = classificador
    cls.fit(X_train, y_train)
    t = time.time()
    aux = classificador.predict(X_test)
    f1score   = f1_score(y_test, aux, average = 'macro')
#     precision = precision_score(y_test, aux, average = 'macro')
#     recall    = recall_score(y_test, aux, average = 'macro')
    acc_train = classificador.score(X_train, y_train)
    acc_test  = classificador.score(X_test, y_test)
    tempoExe  = time.time() - t
        
    return acc_train, acc_test, f1score, tempoExe

**Testando o Conjunto de Teste**

In [9]:
def grid(dataset):
    
    normDic = {'0': 'não aplicado', '1': 'MinMaxScaler', '2': 'StandardScaler',
               '3': 'MaxAbsScaler', '4': 'RobustScaler'}
    extratores = ['AutoColorCorrelogram', 'FCTH', 'Gabor', 'GCH', 'LBP', 'LCH', 
                  'Moments', 'ReferenceColorSimilarity']
    nomes = ['Extrator', 'Normalizador', 'Classificador', 'Tempo', 'Acurácia', 'F1 Score']
    classificadores = [(GaussianNB(), 'Gaussian Naive Bayes'), (LogisticRegression(), 'Logistic Regression'), 
                       (DecisionTreeClassifier(), 'Decision Tree'), (KNeighborsClassifier(n_neighbors = 3), 'KNN'),
                       (LinearDiscriminantAnalysis(), 'Linear Discriminant Analysis'), (SVC(), 'SVM'), 
                       (RandomForestClassifier(random_state=42), 'Random Forest'), (MLPClassifier(alpha=1), 'MLP')]
    
    analise = []
    
    for extrator in extratores:
        
        df = leitura(dataset, extrator)
        
        for norm in range(5):
            X_train, X_test, y_train, y_test = treinoTeste(df)
            X_train, X_test = normalizar(X_train, X_test, norm)
            
            for cls, nome in classificadores:
                dados = treinarClassificadores(cls, X_train, X_test, y_train, y_test)
                acc_train, acc_test, f1score, tempoExe = dados
            
                resultados = [extrator, normDic[str(norm)], nome, tempoExe, acc_test, f1score]
            
                analise.append(resultados)
            
    return pd.DataFrame(analise, columns=nomes).set_index(['Extrator', 'Normalizador', 'Classificador'])

In [10]:
def selecao(resultados):
    
    acc = (resultados['Acurácia'] + resultados['F1 Score']) / 2
    
    return pd.DataFrame(resultados.loc[np.argmax(acc), :]).T

In [15]:
predator = grid('Alien-vs-Predator')

In [23]:
melhorPredator = selecao(predator)

In [24]:
melhorPredator

,,,Tempo,Acurácia,F1 Score
FCTH,não aplicado,MLP,0.004694,0.791367,0.791324


In [18]:
rock = grid('Rock-Paper-Scissors')

In [21]:
melhorRock = selecao(rock)

In [22]:
melhorRock

,,,Tempo,Acurácia,F1 Score
LBP,RobustScaler,MLP,0.008467,0.949772,0.949882


# O porquê
## Alien vs Predator

In [25]:
alien = leitura('Alien-vs-Predator', 'FCTH')

In [45]:
alien = alien.describe().T
alien.head()

,count,mean,std,min,25%,50%,75%,max
1,694.0,2.198847,2.862546,0.0,0.0,0.0,5.0,7.0
2,694.0,3.515850,1.997591,0.0,2.0,3.0,5.0,7.0
3,694.0,5.572046,1.964291,0.0,4.0,7.0,7.0,7.0
4,694.0,0.060519,0.338612,0.0,0.0,0.0,0.0,5.0
5,694.0,0.077810,0.455523,0.0,0.0,0.0,0.0,5.0


In [55]:
np.std(alien['max'] - alien['min'])

2.0651747702306515

FCTH gera características baseado em cor e textura (192 características), as quais apresentam valores desnomalizados. Dataset Alien vs Predator tem diferentes cores e texturas entre as duas classes. Como observado acima, a variação entre as escalas das características são pequenas. Isso pode ter ocasionado a não normalização de nosso dataset

## Rock Paper Scissor

In [30]:
paper = leitura('Rock-Paper-Scissors', 'LBP')

In [47]:
paper = paper.describe().T
paper.head()

,count,mean,std,min,25%,50%,75%,max
1,2188.0,1504.161792,432.404538,483.0,1356.0,1652.0,1809.0,2257.0
2,2188.0,173.488117,62.074474,38.0,151.0,183.0,204.0,442.0
3,2188.0,466.110146,116.830851,128.0,380.0,467.0,542.0,914.0
4,2188.0,262.932815,50.409549,102.0,239.0,269.0,294.0,415.0
5,2188.0,203.561700,84.504891,34.0,148.0,188.0,266.0,458.0


In [54]:
np.std(paper['max'] - paper['min'])

764.9842407099286

O conjunto não apresenta diferença significativa de cores, umas vez que estas são as mesmas. Assim, o melhor extrator para o dataset foi o LBP o qual só gera características baseado na textura. Outra característica dos dados é que após a aplicação do LBP, nossos descritores apresentam bastante variação quanto à escala, assim a aplicação de um normalizador veio a calhar